<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2026-1/Forwards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 1218, done.
remote: Counting objects: 100% (450/450), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 1218 (delta 261), reused 279 (delta 158), pack-reused 768 (from 1)
Receiving objects: 100% (1218/1218), 64.25 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (766/766), done.
Updating files: 100% (453/453), done.


#Funciones auxiliares

In [2]:
import numpy as np

def talamb(nodos, curva, plazos):
    nodos = nodos.flatten()
    curva = curva.flatten()
    plazos = plazos.flatten()

    # Ensure nodos are sorted for np.interp
    sorted_indices = np.argsort(nodos)
    nodos_sorted = nodos[sorted_indices]
    curva_sorted = curva[sorted_indices]

    # Handle edge cases: plazos outside the range of nodos
    TC = np.interp(plazos, nodos_sorted, curva_sorted, left=curva_sorted[0], right=curva_sorted[-1])
    TL = np.interp(plazos, nodos_sorted, curva_sorted, left=curva_sorted[0], right=curva_sorted[-1])

    # Find indices for interpolation within the range
    indices = np.searchsorted(nodos_sorted, plazos) - 1
    indices[indices < 0] = 0
    indices[indices >= len(nodos_sorted) - 1] = len(nodos_sorted) - 2

    nodos_i = nodos_sorted[indices]
    nodos_i_plus_1 = nodos_sorted[indices + 1]
    TC_interp = curva_sorted[indices]
    TL_interp = curva_sorted[indices + 1]

    # Calculate TF using vectorized operations
    TF = np.where(plazos < nodos_sorted[0], curva_sorted[0],
             np.where(plazos > nodos_sorted[-1], curva_sorted[-1],
                      ((((1 + TL_interp * nodos_i_plus_1 / 360) / (1 + TC_interp * nodos_i / 360)) **
                        ((plazos - nodos_i) / (nodos_i_plus_1 - nodos_i)) *
                        (1 + TC_interp * nodos_i / 360)) - 1) * 360 / plazos))

    return TF

## Definición parámetros Python

In [8]:
!pip install yfinance
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

fval = datetime.strptime("20230310", "%Y%m%d") #Fecha de valoración
itpl = 1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
#setwd(direc)
nh = 365*5

# FORWARDS TDC
bext="RiesgosFinancieros/2024-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2024-1/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc=['USDMXN=X']
plazos_fwd=np.array([360, 9, 35, 95, 183])
contratos_fwd=np.array([500, -290, 290, -460, 440])
kst_fwd=[20.5, 19.4, 20.6, 20, 20.5]
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

# FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2024-1/Tarea/tasa_guber.txt"
SymbolsEQ_find=["^MXX"] #tienen que ir en orden alfabético
plazos_fwd_ind=np.array([360, 9, 3500])
contratos_fwd_ind=np.array([5, -2, 5])
kst_fwd_ind=np.array([52000, 49500, 60000])
nominal_fwd_ind=1


# Carga de datos

In [9]:
import pandas as pd

#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1 = pd.read_table(bext,header=None)
data2 = pd.read_table(bdom,header=None)

########minimos para parametrizar
n1 = data1.shape[0]
n2 = data2.shape[0]
m1_ftdc = data1.shape[1]
m2_ftdc = data2.shape[1]
n = min(n1, n2) - 1
###NODOS###
nodos1_ftdc = data1.iloc[0,1:m1_ftdc]
nodos2_ftdc = data2.iloc[0,1:m2_ftdc]
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc = pd.DataFrame(data1.iloc[1:n1,1:m1_ftdc], dtype=float)
x1_ftdc['Date'] = pd.to_datetime(data1.iloc[1:n1,0], format="%Y%m%d")
x2_ftdc = data2.iloc[1:n1,1:m2_ftdc]
x2_ftdc['Date'] = pd.to_datetime(data2.iloc[1:n1, 0], format="%Y%m%d")

if yext==1:
  start_date=fval- timedelta(days=nh)
  start_date = datetime.now().date() - timedelta(days=nh)
  X3_ftdc = yf.download(SymbolsFX_ftdc,start_date)
  X3_ftdc=X3_ftdc.Close.dropna()
  X3_ftdc=X3_ftdc.reset_index()
else:
  data3=pd.read_table(bext,header=None)
  n3 = data3.shape[0]
  m3 = data3.shape[1]
  x3_ftdc = pd.DataFrame(data3.iloc[1:n3,1:m3], dtype=float)
  x3_ftdc['Date'] = pd.to_datetime(data3.iloc[1:n3, 0], format="%Y%m%d")


# CARGA DE DATOS DE  FORWARD DE IPC
data = pd.read_table(base,header=None)
n = data.shape[0]
m_gov=data.shape[1]
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov= pd.DataFrame(data.iloc[1:n,1:m_gov], dtype=float)
x_orig_gov['Date'] = pd.to_datetime(data.iloc[1:n,0], format="%Y%m%d")
nodos_gov=data.iloc[0,1:m_gov]

X3_find = yf.download(SymbolsEQ_find,start_date)
X3_find = X3_find.Close.dropna()
X3_find=X3_find.reset_index()





/tmp/ipython-input-1748235201.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  X3_ftdc = yf.download(SymbolsFX_ftdc,start_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1748235201.py:48: FutureWarning: YF.download() has changed argument auto_adjust default to True
  X3_find = yf.download(SymbolsEQ_find,start_date)
[*********************100%***********************]  1 of 1 completed


## Forwards de Tipo de Cambio

In [21]:
# Filtrar los dataframes por la fecha fval
x1_ftdc_filtered = x1_ftdc[x1_ftdc['Date'] == fval]
x2_ftdc_filtered = x2_ftdc[x2_ftdc['Date'] == fval]
X3_ftdc_filtered = X3_ftdc[X3_ftdc['Date'] == fval]

# Convertir los dataframes filtrados a arrays numpy (excluyendo la columna 'Date')
x1_ftdc_array = x1_ftdc_filtered.iloc[:, :-1].values
x2_ftdc_array = x2_ftdc_filtered.iloc[:, :-1].values
X3_ftdc_array = X3_ftdc_filtered.iloc[:, -1].values # Assuming the last column is the price

m = len(plazos_fwd)

X1_fwtdc = np.zeros((1, m)) # Only one row since we filtered by date
X2_fwtdc = np.zeros((1, m))

def approx(x, y, new_x, rule=2):
    return np.interp(new_x, x, y)

# Convert nodos to numeric, coercing errors
nodos1_ftdc_numeric = pd.to_numeric(nodos1_ftdc, errors='coerce').values
nodos2_ftdc_numeric = pd.to_numeric(nodos2_ftdc, errors='coerce').values


# Since we filtered for a single date, we iterate over the columns
if itpl == 0:
    X1_fwtdc[0, :] = approx(nodos1_ftdc_numeric, x1_ftdc_array[0, :], plazos_fwd)
    X2_fwtdc[0, :] = approx(nodos2_ftdc_numeric, x2_ftdc_array[0, :], plazos_fwd)
else:
    X1_fwtdc[0, :] = talamb(nodos1_ftdc_numeric, x1_ftdc_array[0, :], plazos_fwd) # Pass numpy arrays
    X2_fwtdc[0, :] = talamb(nodos2_ftdc_numeric, x2_ftdc_array[0, :], plazos_fwd) # Pass numpy arrays


if trlib == 1:
    X1_fwtdc[0, :] = ((1 + X1_fwtdc[0, :]) ** (plazos_fwd / 180) - 1) * 360 / plazos_fwd

def futuroTC(t, tl, tn, s, k):
    # Ensure t, tl, tn, s, and k are numpy arrays
    t = np.array(t)
    tl = np.array(tl)
    tn = np.array(tn)
    s = np.array(s)
    k = np.array(k)

    f = s * ((1 + tn * t / 360) / (1 + tl * t / 360))
    return (f - k) / (1 + t * tn / 360)

# Ensure X3_ftdc_array is a numpy array
X3_ftdc_array = np.array(X3_ftdc_filtered.iloc[0, -1]) # Select the price for the filtered date

# Expand X3_ftdc_array to match the shape for calculation
X3_ftdc_expanded = np.full_like(plazos_fwd, X3_ftdc_array)

V0_fwtdc = futuroTC(plazos_fwd, X1_fwtdc[0, :], X2_fwtdc[0, :], X3_ftdc_expanded, kst_fwd) * contratos_fwd * nominal_fwd
print(V0_fwtdc)

[ -374.44249223  -207.9806209    752.79513923 -1440.6165916
   551.52402764]


## FORWARD DE INDICES

In [24]:
# Filtrar los dataframes por la fecha fval
x_orig_gov_filtered = x_orig_gov[x_orig_gov['Date'] == fval]
X3_find_filtered = X3_find[X3_find['Date'] == fval]


m_ind = len(plazos_fwd_ind)

X1_fwind = np.zeros((1, m_ind))  # Matriz para DIVIDENDOS, assuming dividends are zero for now
X2_fwind = np.zeros((1, m_ind))

def approx(x, y, new_x, rule=2):
    return np.interp(new_x, x, y)

# Convert nodos to numeric, coercing errors
nodos_gov_numeric = pd.to_numeric(nodos_gov, errors='coerce').values

# Convert the curve data to numeric, coercing errors
curva_gov_numeric = pd.to_numeric(x_orig_gov_filtered.iloc[0, :-1], errors='coerce').values


# Since we filtered for a single date, we iterate over the columns
if itpl == 0:
    X2_fwind[0, :] = approx(nodos_gov_numeric, curva_gov_numeric, plazos_fwd_ind)
else:
    X2_fwind[0, :] = talamb(nodos_gov_numeric, curva_gov_numeric, plazos_fwd_ind)

# Get the index price for the filtered date
X3_find_price = X3_find_filtered.iloc[0, -1]


V0_fwind = futuroTC(plazos_fwd_ind, X1_fwind[0, :], X2_fwind[0, :], X3_find_price, kst_fwd_ind) * contratos_fwd_ind * nominal_fwd_ind

print(V0_fwind)

[ 25441.92613512  -6780.41728228 137589.19625332]
